In [ ]:
# auto_utterance_action.ipynb
import os
import re
import json
import logging
import warnings


import pandas as pd
import re
from tqdm.auto import tqdm
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate

# Load the Ollama LLM
llm = OllamaLLM(model="llama3.2")

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=[],
    template="""
Generate a unique utterance for an audio engineering task and its corresponding action order. The utterance should be a command or request, and the action order should be a list of steps to accomplish the task.

Format the output as follows:
Utterance: [insert utterance here]
Action Order: ["step 1", "step 2", ...]

Examples:
Utterance: Solo track 1
Action Order: ["Track 1", "Solo"]

Utterance: Apply filter delay
Action Order: ["Search Device", "Filter delay"]

Utterance: Increase Control 1 by 20
Action Order: ["Control 1", "Now", "20"]

Now generate a new, unique utterance and action order:
"""
)

def extract_utterance_and_action(response):
    utterance_match = re.search(r'Utterance: (.+)', response)
    action_order_match = re.search(r'Action Order: (\[.+\])', response)
    
    utterance = utterance_match.group(1) if utterance_match else None
    action_order = eval(action_order_match.group(1)) if action_order_match else None
    
    return utterance, action_order

def generate_utterance_and_action():
    output = llm(prompt_template.format())
    return extract_utterance_and_action(output)

# Generate a specified number of utterances and action orders
num_generations = 100  # You can adjust this number
generated_data = []

for _ in tqdm(range(num_generations), desc="Generating utterances and actions"):
    utterance, action_order = generate_utterance_and_action()
    if utterance and action_order:
        generated_data.append({"Utterance": utterance, "Action_Order": action_order})

# Convert to DataFrame
df = pd.DataFrame(generated_data)

# Display the results
print(df)

# Save the results
df.to_csv("generated_utterances_and_actions.csv", index=False)
df.to_json("generated_utterances_and_actions.json", orient="records")